In [1]:
from geopy.geocoders import Nominatim
import time
from datetime import datetime, timedelta
import requests
import psycopg2 as pg
import pandas.io.sql as psql
import config
from config import epa_api, email
import pandas as pd

In [2]:
#Creating connection to postgress
engine = pg.connect(
    database="postgres",
    user="postgres",
    password="geisadmin01",
    host="geisteam4.coe2ggfhl77s.us-east-1.rds.amazonaws.com",
    port='5432'
)

In [3]:
#Loading in data
air_qual_df  = psql.read_sql('SELECT * FROM air_quality_table', engine)
site_df  = psql.read_sql('SELECT * FROM site_info_table', engine)

In [4]:
air_qual_df.head(10)

,_c0,cbsa_code,date,year,month,day,aqi,category,defining_parameter
0,3362377,21020,2001-12-06,2001,12,6,40,Good,PM2.5
1,3362378,21020,2001-12-09,2001,12,9,35,Good,PM2.5
2,3362379,21020,2001-12-12,2001,12,12,40,Good,PM2.5
3,3362380,21020,2001-12-15,2001,12,15,28,Good,PM2.5
4,3362381,21020,2001-12-18,2001,12,18,15,Good,PM2.5
5,3362382,21020,2001-12-21,2001,12,21,18,Good,PM2.5
6,3362383,21020,2001-12-24,2001,12,24,22,Good,PM2.5
7,3362384,21020,2001-12-27,2001,12,27,47,Good,PM2.5
8,3362385,21020,2001-12-30,2001,12,30,34,Good,PM2.5
9,3366422,22180,2001-01-01,2001,1,1,51,Moderate,PM2.5


In [5]:
site_df.head(10)

,_c0,cbsa_code,date,number_of_sites_reporting,city_ascii,state_id,state_name,lat,lng,population,density
0,3362381,21020,2001-12-18,1,Elizabeth City,NC,North Carolina,36.294201,-76.236000,17725.0,584.0
1,3362382,21020,2001-12-21,1,Elizabeth City,NC,North Carolina,36.294201,-76.236000,17725.0,584.0
2,3366437,22180,2001-01-16,2,Fayetteville,NC,North Carolina,35.084999,-78.977203,333366.0,550.0
3,3366453,22180,2001-02-01,1,Fayetteville,NC,North Carolina,35.084999,-78.977203,333366.0,550.0
4,3366479,22180,2001-02-27,2,Fayetteville,NC,North Carolina,35.084999,-78.977203,333366.0,550.0
5,3366483,22180,2001-03-03,1,Fayetteville,NC,North Carolina,35.084999,-78.977203,333366.0,550.0
6,3366485,22180,2001-03-05,2,Fayetteville,NC,North Carolina,35.084999,-78.977203,333366.0,550.0
7,3366486,22180,2001-03-06,1,Fayetteville,NC,North Carolina,35.084999,-78.977203,333366.0,550.0
8,3366496,22180,2001-03-16,1,Fayetteville,NC,North Carolina,35.084999,-78.977203,333366.0,550.0
9,3366499,22180,2001-03-19,1,Fayetteville,NC,North Carolina,35.084999,-78.977203,333366.0,550.0


In [6]:
def update_sites(site_df):
    #using GEOPY to pull the longitude and latitude for specified cities in site_df
    geolocator = Nominatim(user_agent="AQI_predictor")
    sites = site_df.drop_duplicates(subset = ['city_ascii'])[['city_ascii', 'state_name']]

    for city, state in zip(sites['city_ascii'], sites['state_name']):
        location = geolocator.geocode(city + ', ' + state)

        site_df.loc[site_df['city_ascii'] == city,["lat", "lng"]] =[location.latitude, location.longitude]
    print('-sites updated-')
    return site_df

In [7]:
def updateAirQual(air_df, site_df):
    parameters = {'42101': 'CO', '42602': 'N02', '44201': 'Ozone', '81102':'PM10', '88101':'PM2.5'}
    max_aqi = 0
    defining_param = ''
    record_count = 1
    set_count = 1

    print
    for row in range(15):


        if (row % 10 == 0 and row >= 10):
            set_count += 1
            record_count = 1
            time.sleep(60)


        bdate = edate = air_qual_df.loc[row, 'date'].strftime("%Y%m%d")
        lat, lng = site_df.loc[row, ['lat', 'lng']]

        epa_url = 'https://aqs.epa.gov/data/api/dailyData/byBox?email=' + email + '&key=' + epa_api + \
                '&param=44201,88101,81102,42101,42602&bdate=' + bdate + '&edate=' + edate + '&minlat=' + str(lat-0.25) + \
                '&maxlat=' + str(lat + 0.25) + '&minlon=' + str(lng - 0.25) + '&maxlon=' + str(lng + 0.25)

        print(f'processing record {record_count} of set {set_count} | {site_df.loc[row, "city_ascii"]}, ' \
              f'{air_qual_df.loc[row, "date"]} ')
        try:
            epa_request = requests.get(epa_url).json()

            max_aqi = 0
            defining_param = ''
            for obs in epa_request['Data']:
                if isinstance(obs['aqi'], int):    
                    if obs['aqi'] > max_aqi:
                        max_aqi = obs['aqi']
                        defining_param = parameters[obs['parameter_code']]

            if max_aqi < 50:
                category = 'Good'
            elif max_aqi < 100:
                category = 'Moderate'
            elif max_aqi < 150:
                category = 'Unhealthy for Sensitive Groups'
            elif max_aqi < 200:
                category = 'Unhealthy'
            elif max_aqi < 300:
                category = 'Very Unhealthy'
            else:
                category = 'Hazardous'

            air_qual_df.loc[row,['aqi', 'category', 'defining_param']] = [max_aqi, category, defining_param]
        except:
            print("entry not found, moving on to next one")

        record_count += 1
        time.sleep(5)
               
    print('sets run: ' + str(set_count))
    return air_df
    
    
    

In [8]:
def saveDF(air_df, site_df):
    air_df.to_sql('air_qual_data', engine)
    site_df.to_sql('site_df_data', engine)

In [9]:
test_air = air_qual_df.loc[:15, :]
test_site = site_df.loc[:15, :]

update_sites(test_site)
updateAirQual(test_air, test_site)

E:\Users\speen\anaconda3\envs\mlenv\lib\site-packages\pandas\core\indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


-sites updated-
processing record 1 of set 1 | Elizabeth City, 2001-12-06 
processing record 2 of set 1 | Elizabeth City, 2001-12-09 
processing record 3 of set 1 | Fayetteville, 2001-12-12 
processing record 4 of set 1 | Fayetteville, 2001-12-15 
processing record 5 of set 1 | Fayetteville, 2001-12-18 
processing record 6 of set 1 | Fayetteville, 2001-12-21 
processing record 7 of set 1 | Fayetteville, 2001-12-24 
processing record 8 of set 1 | Fayetteville, 2001-12-27 
processing record 9 of set 1 | Fayetteville, 2001-12-30 
processing record 10 of set 1 | Fayetteville, 2001-01-01 
processing record 1 of set 2 | Fayetteville, 2001-01-02 
processing record 2 of set 2 | Fayetteville, 2001-01-03 
processing record 3 of set 2 | Fayetteville, 2001-01-04 
processing record 4 of set 2 | Fayetteville, 2001-01-05 
processing record 5 of set 2 | Charlotte, 2001-01-06 
sets run: 2


,_c0,cbsa_code,date,year,month,day,aqi,category,defining_parameter
0,3362377,21020,2001-12-06,2001,12,6,40,Good,PM2.5
1,3362378,21020,2001-12-09,2001,12,9,35,Good,PM2.5
2,3362379,21020,2001-12-12,2001,12,12,33,Good,PM2.5
3,3362380,21020,2001-12-15,2001,12,15,33,Good,PM2.5
4,3362381,21020,2001-12-18,2001,12,18,25,Good,PM2.5
5,3362382,21020,2001-12-21,2001,12,21,38,Good,PM2.5
6,3362383,21020,2001-12-24,2001,12,24,27,Good,PM2.5
7,3362384,21020,2001-12-27,2001,12,27,45,Good,PM2.5
8,3362385,21020,2001-12-30,2001,12,30,56,Moderate,PM2.5
9,3366422,22180,2001-01-01,2001,1,1,51,Moderate,PM2.5
